In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [47]:
len(m_svd)

2345796

In [41]:
len(X_test)

2345796

In [42]:
404290+2345796

2750086

In [2]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [3]:
X_train = pd.read_csv("data/train_inter.csv")

X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))

X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())



In [4]:
X_test = pd.read_csv("data/test_inter.csv")

X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [5]:
X = map(lambda x: ' '.join(x), X_train['question1'])
X1 = map(lambda x: ' '.join(x), X_train['question2'])
Y = map(lambda x: ' '.join(x), X_test['question1'])
Y1 = map(lambda x: ' '.join(x), X_test['question2'])
hw = HashingVectorizer(n_features=200).fit(X_train['question1'] + X_train['question2'])


In [6]:
X = pd.DataFrame(hw.transform(X).todense())
X1 = pd.DataFrame(hw.transform(X1).todense())
Y = pd.DataFrame(hw.transform(Y).todense())
Y1 = pd.DataFrame(hw.transform(Y1).todense())

In [7]:
tfidf = feature_extraction.text.TfidfTransformer()
tfidf.fit(X+X1)

X = pd.DataFrame(tfidf.transform(X).todense())
X1 = pd.DataFrame(tfidf.transform(X1).todense())
Y = pd.DataFrame(tfidf.transform(Y).todense())
Y1 = pd.DataFrame(tfidf.transform(Y1).todense())

In [8]:
X_train = pd.concat([X_train, X], axis=1, join_axes=[X_train.index])
del X
X_train = pd.concat([X_train, X1], axis=1, join_axes=[X_train.index])
del X1
X_test = pd.concat([X_test, Y], axis=1, join_axes=[X_test.index])
del Y
X_test = pd.concat([X_test, Y1], axis=1, join_axes=[X_test.index])
del Y1

In [8]:
tmp = np.abs(X - X1)
X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])


In [9]:
tmp = np.abs(Y - Y1)
X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])


In [10]:
cores_dict = pd.read_csv("data/question_max_kcores.csv", index_col="qid").to_dict()["max_kcore"]
def gen_qid1_max_kcore(row):
    return cores_dict[row["qid1"]]
def gen_qid2_max_kcore(row):
    return cores_dict[row["qid2"]]

#def gen_max_kcore(row):
#    return max(row["qid1_max_kcore"], row["qid2_max_kcore"])

X_train["qid1_max_kcore"] = X_train.apply(gen_qid1_max_kcore, axis=1)
#X_test["qid1_max_kcore"] = X_test.apply(gen_qid1_max_kcore, axis=1)
X_train["qid2_max_kcore"] = X_train.apply(gen_qid2_max_kcore, axis=1)
#X_test["qid2_max_kcore"] = X_test.apply(gen_qid2_max_kcore, axis=1)

In [11]:
q_test = pd.read_csv("data/kcore_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [13]:
X_test[:2]

,test_id,question1,question2,word_match,tfidf_word_match,0,1,2,3,4,...,192,193,194,195,196,197,198,199,qid1_max_kcore,qid2_max_kcore
0,0,"[how, does, the, surface, pro, himself, 4, com...","[why, did, microsoft, choose, core, m3, and, n...",0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0,0
1,1,"[should, i, have, a, hair, transplant, at, age...","[how, much, cost, does, hair, transplant, requ...",0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.30673,0.0,0.0,0,0


In [12]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

del X_test['question1']
del X_test['question2']
del X_test['test_id']

In [15]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,192,193,194,195,196,197,198,199,qid1_max_kcore,qid2_max_kcore
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,0.156312,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0,0
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,0.037057,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.30673,0.0,0.0,0,0


In [28]:
X_train.ix[:2, 202:402]

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.119495,...,0.0,0.0,0.325385,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.131759,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.167796,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X_train.ix[:2, 2:202]

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.113341,...,0.0,0.0,0.308628,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.165352,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.209907,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.380265,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.186087,0.0,0.0,0.0


In [85]:
del X_train

In [13]:
#X_train = pd.read_csv("data/train_inter.csv")
q_train = pd.read_csv("data/freq_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

f_train = pd.read_csv("data/train_features.csv", encoding = "ISO-8859-1")
f_train = f_train.ix[:, 2:]
X_train = pd.concat([X_train, f_train], axis=1)

In [14]:
q_train = pd.read_csv("data/f1_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [15]:
q_train = pd.read_csv("data/train_lda_feat.csv", encoding = "ISO-8859-1")
q_train = q_train.ix[:, 6:]
X_train = pd.concat([X_train, q_train], axis=1)

In [16]:
q_train = pd.read_csv("stacking/nn_21.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [17]:
q_train = pd.read_csv("data/pagerank_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)
#q_train[:2]

In [18]:
q_train = pd.read_csv("data/jaccard_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [19]:
q_train = pd.read_csv("stacking/nn_21_lstm1.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [24]:
X_train[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda,lstm,q1_pr,q2_pr,jaccard,lstm1
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.266667,0.137590,0.127564,0.134090,0.588275,1.608480e-07,3.047631e-07,1.0,0.329186
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,5.400000,0.538923,0.423051,0.812442,0.203615,8.198776e-07,3.097268e-07,1.0,0.280913


In [129]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,kur_q2vec,q1_q2_intersect,cosine_distance_lda,cityblock_distance_lda,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda,lstm
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,0.156312,0.0,0.0,...,-0.144866,0,0.573341,1.160190,1.0,7.833784,0.399663,0.290631,0.65838,0.102577
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,0.037057,0.0,0.0,...,-0.042937,0,0.251020,0.615409,1.0,3.595264,0.265033,0.210156,0.35913,0.310506


In [82]:
del X, X1

In [21]:
import gc
gc.collect()

158

In [25]:
q_test = pd.read_csv("data/freq_test.csv")


In [26]:
X_test = pd.concat([X_test, q_test], axis=1)

In [27]:
f_test = pd.read_csv("data/test_features.csv", encoding = "ISO-8859-1")
f_test = f_test.ix[:, 2:]

In [28]:
X_test = pd.concat([X_test, f_test], axis=1)

In [29]:
q_test = pd.read_csv("data/f1_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [30]:
q_test = pd.read_csv("data/test_lda_feat.csv", encoding = "ISO-8859-1")
q_test = q_test.ix[:, 3:]
X_test = pd.concat([X_test, q_test], axis=1)

In [31]:
q_test = pd.read_csv("stacking/nn_21_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [32]:
q_test = pd.read_csv("data/pagerank_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [33]:
q_test = pd.read_csv("data/jaccard_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [34]:
q_test = pd.read_csv("stacking/nn_21_lstm1_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [35]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda,lstm,q1_pr,q2_pr,jaccard,lstm1
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,0.156312,0.0,0.0,...,1.0,7.833784,0.399663,0.290631,0.65838,0.102577,2.088105e-07,2.088105e-07,1.0,0.075481
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,0.037057,0.0,0.0,...,1.0,3.595264,0.265033,0.210156,0.35913,0.310506,2.368239e-07,2.194807e-07,1.0,0.413205


In [36]:
X_train[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda,lstm,q1_pr,q2_pr,jaccard,lstm1
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.266667,0.137590,0.127564,0.134090,0.588275,1.608480e-07,3.047631e-07,1.0,0.329186
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,5.400000,0.538923,0.423051,0.812442,0.203615,8.198776e-07,3.097268e-07,1.0,0.280913


In [246]:
X_train1 = X_train

In [254]:
X_train1[:3]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,cityblock_distance_lda,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda,lstm,q1_pr,q2_pr,lstm1
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.232798,1.0,1.266667,0.137590,0.127564,0.134090,0.588275,1.608480e-07,3.047631e-07,0.329186
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,1.241746,1.0,5.400000,0.538923,0.423051,0.812442,0.203615,8.198776e-07,3.097268e-07,0.280913
2,0.363636,0.355191,0.0,0.0,0.0,0.0,0.0,0.167796,0.0,0.0,...,1.089333,1.0,6.333333,0.411444,0.304798,0.655172,0.476624,3.049200e-07,1.609126e-07,0.677454


In [37]:
X_train = X_train.replace([np.inf, -np.inf], np.nan)

In [38]:
X_train= X_train.fillna(X_train.mean())

In [39]:
X_train=X_train.astype(np.float32)

In [40]:
X_train.isnull().sum().sum()

0

In [41]:
X_test = X_test.replace([np.inf, -np.inf], np.nan)

In [42]:
X_test= X_test.fillna(X_test.mean())

In [43]:
X_test=X_test.astype(np.float32)

In [44]:
X_test.isnull().sum().sum()

0

In [180]:
del X_test['jaccard']
del X_train['jaccard']

In [63]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.07
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 2000

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.05,
    'min_child_weight': 50,
    #'max_depth': 7
    #'num_class': 3
    
}

pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1
        
        #xgboost
        '''
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)

        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        '''
        '''
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=2000, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]
        
        
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
        ext = sklearn.linear_model.LogisticRegression()
        ext.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = ext.predict_proba(X_train.ix[ite, :])[:, 1]
    #ypred = sum(ypred) / len(ypred) 0.401408 0.392476
    pred_train[ite] = ypred

    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGB: 0.281 +- 0.003


In [ ]:
0.268
0.294
0.323

In [64]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [65]:
pred_train = foo(pred_train)

In [66]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['y']
pred_train.to_csv("stacking/lin_30.csv", index=None)

In [271]:
del Xdatatest2

In [50]:
import gc
gc.collect()

165

In [58]:
submit = pd.read_csv("data/sample_submission.csv")

In [56]:
#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 100,
    #'max_depth': 6
    #'num_class': 3
    
}

In [188]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=2000, seed=0, **params)
lgb.fit(X_train, y_train)


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.05, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=50,
        min_split_gain=0, n_estimators=2000, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [41]:
    ext = sklearn.ensemble.ExtraTreesClassifier(n_estimators=1000,
                                                    criterion='gini',
                                                    max_depth=30,
                                                    min_samples_split=2,
                                                    min_samples_leaf=1,
                                                    min_weight_fraction_leaf=0.0,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_split=1e-07,
                                                    bootstrap=False,
                                                    oob_score=False,
                                                    n_jobs=-1,
                                                    random_state=None,
                                                    verbose=0,
                                                    warm_start=False,
                                                    class_weight=None)
ext.fit(X_train, y_train)

/home/fao3864/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=30, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=-1, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [51]:
        ext = sklearn.ensemble.RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=20)
        ext.fit(X_train, y_train)


/home/fao3864/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [67]:
        ext = sklearn.linear_model.LogisticRegression()
        ext.fit(X_train, y_train)

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [68]:
test_pred1 = ext.predict_proba(X_test.ix[:1000000])[:, 1]

In [69]:
test_pred2 = ext.predict_proba(X_test.ix[1000001:])[:, 1]

In [71]:
test_pred1[:4]

array([ 0.04500053,  0.28164112,  0.40565882,  0.00154451])

In [72]:
test_pred1[-4:]

array([ 0.01236717,  0.07248959,  0.83883488,  0.03395509])

In [73]:
test_pred2[:4]

array([ 0.09783528,  0.00108652,  0.01733689,  0.03079797])

In [151]:
test_pred2[-4:]

array([  1.67587699e-03,   2.31711933e-04,   3.72899384e-01,
         3.89707027e-01])

In [148]:
len(test_pred1)

1000001

In [152]:
len(test_pred2)

1345795

In [74]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [47]:
len(test_pred)

2345796

In [195]:
test_pred

array([  3.44825252e-03,   2.31987433e-01,   5.78899062e-01, ...,
         8.52588377e-05,   2.30536292e-01,   4.70305114e-01])

In [163]:
del Xdatatrain

In [48]:
import gc
gc.collect()

19

In [38]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 2000

In [155]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)

In [156]:
Xdataval = xgb.DMatrix(data=X_train.ix[:10, :].values,
                                     label=y_train.ix[:10].values)

In [168]:
del Xdatatest1

In [170]:
Xdatatest2 = xgb.DMatrix(data=X_test.ix[1000001:, :].values)

In [165]:
Xdatatest1 = xgb.DMatrix(data=X_test.ix[:1000000, :].values)

In [157]:

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]

In [158]:
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


[0]	train-logloss:0.647789	eval-logloss:0.65296
[15]	train-logloss:0.345185	eval-logloss:0.371863
[30]	train-logloss:0.267959	eval-logloss:0.298524
[45]	train-logloss:0.242341	eval-logloss:0.274513
[60]	train-logloss:0.231946	eval-logloss:0.250854
[75]	train-logloss:0.226236	eval-logloss:0.240165
[90]	train-logloss:0.220925	eval-logloss:0.231341
[105]	train-logloss:0.216633	eval-logloss:0.232251
[120]	train-logloss:0.213026	eval-logloss:0.225941
[135]	train-logloss:0.210134	eval-logloss:0.218171
[150]	train-logloss:0.207633	eval-logloss:0.214139
[165]	train-logloss:0.20549	eval-logloss:0.212049
[180]	train-logloss:0.203296	eval-logloss:0.211795
[195]	train-logloss:0.201364	eval-logloss:0.207569
[210]	train-logloss:0.199599	eval-logloss:0.206513
[225]	train-logloss:0.198068	eval-logloss:0.207797
[240]	train-logloss:0.19681	eval-logloss:0.208231
[255]	train-logloss:0.195652	eval-logloss:0.208948
[270]	train-logloss:0.194361	eval-logloss:0.209715
[285]	train-logloss:0.192768	eval-logloss:

In [171]:
test_pred2 = bst.predict(Xdatatest2)


In [167]:
test_pred1[:4]

array([  3.22814728e-03,   3.37183297e-01,   5.60726404e-01,
         9.69345274e-05], dtype=float32)

In [50]:
len(test_pred1)

1000001

In [53]:
test_pred1[:4]

array([  2.29527755e-03,   5.45756638e-01,   6.37812853e-01,
         4.23383579e-04], dtype=float32)

In [172]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [173]:
len(test_pred)

2345796

In [75]:
test_pred = foo(test_pred)

In [76]:
submit['is_duplicate'] = test_pred

In [77]:
submit.to_csv("stacking/lin_30_test.csv", index=None)

In [79]:
submit.mean()

test_id         1.172898e+06
is_duplicate    7.466991e-02
dtype: float64

In [200]:
submit

,test_id,is_duplicate
0,0,0.001163
1,1,0.092256
2,2,0.316260
3,3,0.000180
4,4,0.089728
5,5,0.000924
6,6,0.996947
7,7,0.564485
8,8,0.329441
9,9,0.000331
